In [12]:
import torch
import gp_math_func as MF
n_time = 200
n_stock = 100
n_factor = 10
x = torch.rand(n_time , n_stock , n_factor)
y = torch.rand(n_time , n_stock , 1)
nan = (y > 0.9) + (y < 0.1)
y[nan] = torch.nan
univ = ~nan.all(-1)
# mat = a.T.mm(a)

In [16]:
labels.shape

torch.Size([200, 100, 1])

In [19]:
gp_factor = x
labels = y
insample = torch.arange(n_time) <160
ir_window = 10
n_time , n_stock , n_factor = gp_factor.shape

weight_scheme = ['ew' , 'ic' , 'ir']
window_type   = ['rolling' , 'full' , 'insample'] 
weight_decay  = ['constant' , 'linear' , 'exp']

rankic = torch.full((n_time , n_factor) , fill_value=torch.nan).to(labels)
for i_factor in range(n_factor):
    rankic[:,i_factor] = MF.rankic_2d(gp_factor[...,i_factor] , labels.squeeze(-1) , dim = 1 , universe = univ , min_coverage = 0.)
rankir = MF.ma(rankic , ir_window) / MF.ts_stddev(rankic , ir_window)



In [20]:
rankir

tensor([[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        ...,
        [-0.5019, -0.1219, -0.3724,  ...,  0.3048, -0.5996, -0.6442],
        [-0.5466,  0.0083, -0.3511,  ...,  0.1030, -0.4369, -0.4297],
        [-0.2677, -0.0765, -0.3607,  ..., -0.0181, -0.2699, -0.1135]])

In [ ]:


if weight_scheme == 'ew':
    multifactor = MF.zscore(gp_factor.nanmean(-1),-1)
else:
    # rankic first
    metric_full = torch.zeros(len(labels),n_factor).to(labels)
    for i_factor in range(n_factor):
        rankic = MF.rankic_2d(gp_factor[...,i_factor] , labels , dim = 1 , universe = univ , min_coverage = 0.)
        metric_full[:,i_factor] = rankic
    if weight_scheme == 'ir': metric_full = MF.ts_zscore(metric_full , ir_window)
    
    multifactor = torch.zeros_like(gp_factor[...,0])
    ts_weight = FF.decay_weight(weight_decay , len(multifactor) , exp_halflife=exp_halflife)
    for i in range(len(multifactor)):
        if i < 10: continue
        d = min(window_len , i) if window_type == 'rolling' else i
        f_weight = ts_weight[:d].reshape(1,-1) @ metric_full[i-d:i]
        factor_of_i = gp_factor[i] @ f_weight.reshape(-1,1)
        multifactor[i] = factor_of_i.flatten()

multifactor = pd.DataFrame(multifactor.cpu().numpy() , columns = df_columns , index = df_index)



In [153]:
top_svd_factors(mat1 , a , top_ratio=1.).shape

torch.Size([200, 100, 10])

In [10]:
mat = mat1
svd = torch.linalg.svd(mat)
svd_U = svd.U
svd_U  = torch.ones(n_fac).diag()
vector = torch.einsum('ij,klj->kli', svd_U, a)

In [133]:
svd.S

tensor([51783.1055,  1710.3624,  1701.4822,  1689.5823,  1677.6132,  1672.1703,
         1645.0459,  1639.3440,  1621.0225,  1608.4126])